<a href="https://colab.research.google.com/github/sarathsoundar/Pyspark_MLlib_Learnings/blob/main/MLlib/Linear%20regression/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install pyspark

In [30]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [31]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [32]:
data = spark.read.csv("/content/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [33]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [34]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [35]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [36]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [37]:
output = assembler.transform(data)

In [38]:
output.select("features").show(5)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
+--------------------+
only showing top 5 rows



In [39]:
output.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [40]:
final_data = output.select("features",'Yearly Amount Spent')

In [41]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [42]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [43]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [44]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [45]:
lrModel = lr.fit(train_data)

In [46]:
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.678027414725634,38.260846190142125,0.4440955087133715,62.037882487810904] Intercept: -1046.8369387592113


In [47]:
test_results = lrModel.evaluate(test_data)

In [48]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------+
|          residuals|
+-------------------+
|-3.1950096565037143|
|  6.709286579798345|
|  4.792553860176554|
| -5.769455827825425|
|-21.209566678344004|
|-3.1271336886009067|
|  22.24892045603883|
| 0.6077018481864798|
| -8.395555744883154|
| -5.486787658543449|
| -8.229830935429106|
|-13.897089889052495|
| 18.328757663469162|
|-0.5709458612112712|
| -7.374125750102053|
|  7.432750462444062|
|  -8.38250076211392|
| 1.5547589791210612|
|  9.083152835581643|
|-0.5674590280697203|
+-------------------+
only showing top 20 rows

RMSE: 10.165536332876547


In [49]:
unlabeled_data = test_data.select('features')

In [50]:
predictions = lrModel.transform(unlabeled_data)

In [51]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...| 470.6969100834933|
|[30.9716756438877...| 487.9293231770944|
|[31.0472221394875...|387.70484532884484|
|[31.0613251567161...|  493.324913885727|
|[31.1239743499119...| 508.1566205181098|
|[31.2681042107507...|426.59766686242483|
|[31.2834474760581...| 569.5321689696286|
|[31.3895854806643...| 409.4619092117964|
|[31.4474464941278...|426.99829784010717|
|[31.5257524169682...|449.45241446842533|
|[31.5261978982398...| 417.3243571277669|
|[31.5741380228732...| 558.3063620496393|
|[31.6098395733896...|  426.216791987639|
|[31.6610498227460...| 416.9292994411121|
|[31.7207699002873...|  546.149059228125|
|[31.8209982016720...| 417.2425305507693|
|[31.8279790554652...|448.38524830905544|
|[31.8293464559211...| 383.5975790088539|
|[31.8512531286083...|463.90909383121675|
|[31.8627411090001...| 556.8656002021164|
+--------------------+------------

In [52]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.165536332876547
MSE: 103.33812893503315
